In [ ]:
import wbdata
from datetime import datetime as dt
import pandas as pd
import MySQLdb as myDB
from bs4 import BeautifulSoup as bs
import urllib2
import re

## Get World Bank Country Data

To make up fo missing data, the World Bank data is averaged from 2009 to 2012. Regions are removed from the data.

In [ ]:
df = wbdata.get_dataframe(indicators = {"VC.IHR.PSRC.P5":"homicides", "SL.UEM.TOTL.ZS":"unemployment"}, data_date = dt(2012,1,1)                         )
real_countries = df.index[34:].values

#### wb_datatframe averages indicators over a select number of years

In [ ]:
def wb_dataframe(indicator, years):
    "creates dataframe for indicator over a specified range of years"
    column = wbdata.get_data(indicator, pandas=True, data_date = (dt(years[0],1,1),dt(years[1],1,1)))
    column = column.unstack(1)
    column = column.ix[real_countries,:]
    column = column.mean(1)
    return column

In [ ]:
inds = {"homicides":"VC.IHR.PSRC.P5","income_highest_10":"SI.DST.10TH.10",
        "water_urban":"SH.H2O.SAFE.UR.ZS", "internet_users":"IT.NET.USER.P2",
        "population_total":"SP.POP.TOTL","land_area":"AG.LND.TOTL.K2",
        "net_migration":"SM.POP.NETM",        
       }

dat = {}
for key,value in inds.iteritems():
     dat[key]= wb_dataframe(value,[2009,2012])

world_bank = pd.DataFrame(dat.values(), index = dat.keys()).transpose()

In [ ]:
world_bank

# Get Avdvanced/Emerging Designations from IMF

In [ ]:
def imf_groups(site,table_num,data_name):
    url = urllib2.urlopen(site)
    soup = bs(url,'lxml')
    tab = soup.findAll("table")[table_num]
    tab = [i.get_text() for i in tab.findAll("p")]
    tab = ''.join(tab)
    tab = tab.splitlines()
    tab = filter(None,tab)
    finis = {tab[0]:tab[1:]}
    imf_ser = pd.Series([data_name for i in finis.values()[0]],
              index = finis.values()[0])
    return imf_ser


In [ ]:
s3 ="http://www.imf.org/external/pubs/ft/weo/2015/01/weodata/groups.htm"

adv_econ = imf_groups(s3, 4,"adv")

emer_econ = imf_groups(s3,9,"emer")

In [ ]:
imf_list = adv_econ.append(emer_econ)

In [ ]:
#imf_list.index = imf_list.index.str.replace(r"\xf4","o")

In [ ]:
#imf_list.index = imf_list.index.str.replace(r"S\xe3o Tom\xe9 and Pr\xedncipe","Sao Tome and Principe")

In [ ]:
imf_grade = pd.DataFrame(imf_list,columns = ["imf_grade"])

In [ ]:
imf_grade.index.name = "country"

In [ ]:
imf_grade

# Create column for country regions 

In [ ]:
site1 = "https://meta.wikimedia.org/wiki/List_of_countries_by_regional_classification"
soup2 = bs(urllib2.urlopen(site1),'lxml')
tab2 = soup2.find("table",{"class":"wikitable sortable"})

In [ ]:
country = []
region = []
global_south = []

for row in tab2.find_all('tr')[1:]:
    col = row.find_all('td')
    
    column_1 = col[0].string.strip()
    country.append(column_1)
    
    column_2 = col[1].string.strip()
    region.append(column_2)
    
    column_3 = col[2].string.strip()
    global_south.append(column_3)

In [ ]:
columns = {'country':country, 'region':region, 'global_south':global_south    
}

In [ ]:
region_dat = pd.DataFrame(columns)

In [ ]:
region_dat

# Load 3 Dataframes into mysql database

In [ ]:
#Inserts dataframe into MySQL
con = myDB.connect(host='localhost', user='root', passwd='root')
cursor = con.cursor()
cursor.execute("CREATE DATABASE IF NOT EXISTS wb_homicide;")
con.select_db("wb_homicide")

In [ ]:
world_bank.to_sql(con = con, flavor="mysql", name = "wb", if_exists="replace")

imf_grade.to_sql(con = con, flavor="mysql", name = "imf", if_exists="replace")

region_dat.to_sql(con = con, flavor="mysql", name = "region_dat", if_exists="replace")

In [ ]:
cursor.execute("SELECT * FROM wb")
cursor.fetchall()